In [394]:
import torch
import time
import torch.nn.functional as F


n, p = 128, 4000
col = 0

a = torch.randn(100, 16) * 0.5
b = torch.randn(500, 16) * 0.5

match_ab = torch.matmul(a, torch.t(b))
match_ba = torch.t(match_ab)
match_bb = torch.matmul(b, torch.t(b))

p_ab = F.softmax(match_ab, dim=1)
p_bb = F.softmax(match_bb, dim=1)
p_ba = F.softmax(match_ba, dim=1)

In [395]:
match_ba_bb = torch.cat([match_ba, match_bb], dim=1)
p_ba_bb = torch.clamp(F.softmax(match_ba_bb, dim=1), min=1e-8)

In [396]:
N = a.shape[0]
M = b.shape[0]
Tbar_ul, Tbar_uu = p_ba_bb[:, :N], p_ba_bb[:, N:]

In [430]:
middle = torch.inverse(torch.eye(Tbar_uu.shape[1]) - Tbar_uu)# - torch.eye(Tbar_uu.shape[1])

In [433]:
middle = torch.eye(Tbar_uu.shape[1])
for i in range(1, 3):
    middle += torch.matrix_power(Tbar_uu, i)
middle /= Tbar_uu.sum(1, keepdim=True)

In [434]:
p_aba = torch.matmul(torch.matmul(p_ab, middle), Tbar_ul)

In [432]:
p_aba

tensor([[0.0115, 0.0085, 0.0108,  ..., 0.0093, 0.0088, 0.0090],
        [0.0093, 0.0101, 0.0103,  ..., 0.0093, 0.0085, 0.0092],
        [0.0094, 0.0082, 0.0150,  ..., 0.0095, 0.0084, 0.0092],
        ...,
        [0.0094, 0.0086, 0.0111,  ..., 0.0118, 0.0089, 0.0093],
        [0.0094, 0.0083, 0.0103,  ..., 0.0094, 0.0102, 0.0089],
        [0.0089, 0.0083, 0.0103,  ..., 0.0090, 0.0081, 0.0117]])

In [437]:
p_aba / p_aba.sum(1, keepdim=True)

tensor([[0.0151, 0.0088, 0.0112,  ..., 0.0096, 0.0092, 0.0085],
        [0.0094, 0.0128, 0.0100,  ..., 0.0097, 0.0083, 0.0090],
        [0.0097, 0.0081, 0.0217,  ..., 0.0103, 0.0081, 0.0089],
        ...,
        [0.0096, 0.0090, 0.0118,  ..., 0.0157, 0.0092, 0.0091],
        [0.0098, 0.0083, 0.0100,  ..., 0.0099, 0.0126, 0.0081],
        [0.0084, 0.0082, 0.0101,  ..., 0.0090, 0.0074, 0.0153]])

In [165]:
match_ab = torch.matmul(a, torch.t(b))
p_ab = F.softmax(match_ab, dim=1)

match_ba = torch.t(match_ab)
match_bb = torch.matmul(b, torch.t(b))
p_bb = torch.softmax(match_bb, dim=1)
p_ba = F.softmax(match_ba, dim=1)

match_ba_bb = torch.cat([match_ba, match_bb], dim=1)
p_ba_bb = torch.clamp(F.softmax(match_ba_bb, dim=1), min=1e-8)
N = a.shape[0]
M = b.shape[0]
Tbar_ul, Tbar_uu = p_ba_bb[:, :N], p_ba_bb[:, N:]
I = torch.eye(M)
I = I.cuda() if Tbar_uu.is_cuda else I
# middle = torch.inverse(I - Tbar_uu + 1e-8)
middle = torch.inverse(match_bb + 1e-8)
# middle = torch.inverse(I - p_bb + 1e-8)
p_aba = torch.matmul(torch.matmul(p_ab, middle), Tbar_ul)
p_aba /= p_aba.sum(1, keepdim=True)
# p_aba = torch.matmul(torch.matmul(p_ab, p_bb), p_ba)

In [162]:
p_aba.sum(1)

tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000])

In [163]:
middle

tensor([[ 1.2201, -0.3730,  0.6471,  ..., -0.3973, -0.3934, -0.2369],
        [-2.3164,  0.3392, -4.5347,  ...,  0.9172,  2.8041, -0.4114],
        [ 3.0603,  0.5993,  3.7068,  ..., -0.6661, -5.3733,  2.5896],
        ...,
        [ 2.4616,  0.9285,  3.8963,  ..., -0.7809, -5.4913,  1.0392],
        [-1.4368, -0.3912, -3.5948,  ...,  0.0509,  3.4498, -0.8072],
        [-1.6878, -0.0854, -2.4238,  ...,  0.5706,  0.5472, -1.4204]])

In [128]:
p_aba

tensor([[6.0889, 6.0888, 6.0887,  ..., 6.0888, 6.0890, 6.0888],
        [6.0889, 6.0888, 6.0887,  ..., 6.0888, 6.0890, 6.0888],
        [6.0889, 6.0888, 6.0887,  ..., 6.0888, 6.0890, 6.0888],
        ...,
        [6.0889, 6.0888, 6.0887,  ..., 6.0888, 6.0890, 6.0888],
        [6.0889, 6.0888, 6.0887,  ..., 6.0888, 6.0890, 6.0888],
        [6.0889, 6.0888, 6.0887,  ..., 6.0888, 6.0890, 6.0888]])

In [123]:
p_bb

tensor([[0.0003, 0.0002, 0.0003,  ..., 0.0003, 0.0002, 0.0002],
        [0.0002, 0.0003, 0.0003,  ..., 0.0002, 0.0003, 0.0003],
        [0.0003, 0.0003, 0.0003,  ..., 0.0002, 0.0002, 0.0003],
        ...,
        [0.0003, 0.0002, 0.0002,  ..., 0.0003, 0.0002, 0.0003],
        [0.0002, 0.0003, 0.0002,  ..., 0.0002, 0.0003, 0.0002],
        [0.0002, 0.0003, 0.0003,  ..., 0.0003, 0.0002, 0.0003]])

In [ ]:
p_ab

In [113]:
middle

tensor([[1.0100, 0.0100, 0.0100,  ..., 0.0100, 0.0100, 0.0100],
        [0.0100, 1.0100, 0.0100,  ..., 0.0100, 0.0100, 0.0100],
        [0.0100, 0.0100, 1.0100,  ..., 0.0100, 0.0100, 0.0100],
        ...,
        [0.0100, 0.0100, 0.0100,  ..., 1.0100, 0.0100, 0.0100],
        [0.0100, 0.0100, 0.0100,  ..., 0.0100, 1.0100, 0.0100],
        [0.0100, 0.0100, 0.0100,  ..., 0.0100, 0.0100, 1.0100]])

In [114]:
Tbar_ul

tensor([[0.0002, 0.0002, 0.0002,  ..., 0.0002, 0.0002, 0.0002],
        [0.0002, 0.0002, 0.0002,  ..., 0.0002, 0.0002, 0.0002],
        [0.0002, 0.0002, 0.0002,  ..., 0.0002, 0.0002, 0.0002],
        ...,
        [0.0002, 0.0002, 0.0002,  ..., 0.0002, 0.0002, 0.0002],
        [0.0002, 0.0002, 0.0002,  ..., 0.0002, 0.0002, 0.0002],
        [0.0002, 0.0002, 0.0002,  ..., 0.0002, 0.0002, 0.0002]])

In [115]:
p_aba

tensor([[0.0100, 0.0100, 0.0100,  ..., 0.0100, 0.0100, 0.0100],
        [0.0100, 0.0100, 0.0100,  ..., 0.0100, 0.0100, 0.0100],
        [0.0100, 0.0100, 0.0100,  ..., 0.0100, 0.0100, 0.0100],
        ...,
        [0.0100, 0.0100, 0.0100,  ..., 0.0100, 0.0100, 0.0100],
        [0.0100, 0.0100, 0.0100,  ..., 0.0100, 0.0100, 0.0100],
        [0.0100, 0.0100, 0.0100,  ..., 0.0100, 0.0100, 0.0100]])

In [116]:
p_aba.sum(1, keepdim=True)

tensor([[0.9984],
        [0.9984],
        [0.9984],
        [0.9984],
        [0.9984],
        [0.9984],
        [0.9984],
        [0.9984],
        [0.9984],
        [0.9984],
        [0.9984],
        [0.9984],
        [0.9984],
        [0.9984],
        [0.9984],
        [0.9984],
        [0.9984],
        [0.9984],
        [0.9984],
        [0.9984],
        [0.9984],
        [0.9984],
        [0.9984],
        [0.9984],
        [0.9984],
        [0.9984],
        [0.9984],
        [0.9984],
        [0.9984],
        [0.9984],
        [0.9984],
        [0.9984],
        [0.9984],
        [0.9984],
        [0.9984],
        [0.9984],
        [0.9984],
        [0.9984],
        [0.9984],
        [0.9984],
        [0.9984],
        [0.9984],
        [0.9984],
        [0.9984],
        [0.9984],
        [0.9984],
        [0.9984],
        [0.9984],
        [0.9984],
        [0.9984],
        [0.9984],
        [0.9984],
        [0.9984],
        [0.9984],
        [0.9984],
        [0

In [96]:
(p_aba / p_aba.sum(1, keepdim=True))

tensor([[0.5052, 0.4948],
        [0.5018, 0.4982]])

In [55]:
match_ab = torch.matmul(a, torch.t(b))
p_ab = F.softmax(match_ab, dim=1)
p_ba = F.softmax(torch.t(match_ab), dim=1)
p_aba = torch.matmul(p_ab, p_ba)

In [56]:
p_aba.sum(1)

tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000])

In [21]:
X = torch.randn([n, p]) * 0.5
# X = torch.softmax(X, 0)
B = torch.matmul(X.T, X)
B = B.cuda()
B = B.cpu()
now = time.time()
B = torch.inverse(B)
time.time() - now

1.1735286712646484

In [7]:
new_col = v = torch.randn([n, 1]) * 0.5
# new_col = v = torch.softmax(torch.randn([n, 1]) * 0.5, 0)

In [8]:
def add_new_col_to_inverse(old_inverse, new_col):
    B = old_inverse
    v = new_col
    
    u1 = torch.matmul(X.T, v)
    u2 = torch.matmul(B, u1)
    F22inv = 1. / (torch.matmul(v.T, v) - torch.matmul(u1.T, u2))
    u3 = F22inv * u2
    F11inv = B + F22inv * u2 * u2.T
    Bnew = torch.cat([
        torch.cat([F11inv, -u3], 1),
        torch.cat([-u3.T, F22inv], 1)
    ], 0)
    return Bnew

In [9]:
now = time.time()
add_new_col_to_inverse(B, v)
time.time() - now

0.338165283203125

In [29]:
Xnew = torch.cat([X, v], 1)
torch.inverse(torch.matmul(Xnew.T, Xnew))

tensor([[ 1.2506, -0.6256,  0.1035,  1.0205],
        [-0.6256,  0.8223,  0.1486, -0.5141],
        [ 0.1035,  0.1486,  0.4402,  0.3598],
        [ 1.0205, -0.5141,  0.3598,  2.0012]])